<a href="https://colab.research.google.com/github/nidhib08/C4_Grp5_Sign_Language_Detection/blob/main/Sign_Language_Detection_AIML_MiniProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Downloading Necessary Libraries

In [ ]:
!pip install tensorflow keras opencv-python pyttsx3 mediapipe
print("✅ MediaPipe Hands is ready and working correctly!")



✅ MediaPipe Hands is ready and working correctly!


Downloading appropriate versions of libraries

In [ ]:
!pip install numpy==1.26.4 --force-reinstall --no-deps --quiet

!pip uninstall -y numpy
!pip install numpy==1.26.4 --quiet




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 88.6 MB/s eta 0:00:00
Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
shap 0.50.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
pytensor 2.35.1 requires numpy>=2.0, but you have numpy 1

In [ ]:
!pip uninstall -y mediapipe
!pip install mediapipe==0.10.11 --quiet


Found existing installation: mediapipe 0.10.14
Uninstalling mediapipe-0.10.14:
  Successfully uninstalled mediapipe-0.10.14
ERROR: Could not find a version that satisfies the requirement mediapipe==0.10.11 (from versions: 0.10.13, 0.10.14, 0.10.15, 0.10.18, 0.10.20, 0.10.21)
ERROR: No matching distribution found for mediapipe==0.10.11


Checking if Mediapipe is installed successfully

In [ ]:
!pip install mediapipe==0.10.9 --quiet
import types, mediapipe
mediapipe.tasks = types.SimpleNamespace()   # Prevent audio import errors
import mediapipe as mp

print("✅ MediaPipe installed and working!")



ERROR: Could not find a version that satisfies the requirement mediapipe==0.10.9 (from versions: 0.10.13, 0.10.14, 0.10.15, 0.10.18, 0.10.20, 0.10.21)
ERROR: No matching distribution found for mediapipe==0.10.9
✅ MediaPipe installed and working!


Installing Libraries

In [ ]:
pip install opencv-python mediapipe tensorflow scikit-learn ipywidgets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mediapipe to determine which version is compatible with other requirements. This could take a while.
  Using cached mediapipe-0.10.14-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.7 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.7 MB/s eta 0:00:00
Using cached mediapipe-0.10.14-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (35.7 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 50.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yd

In [ ]:
!pip install mediapipe==0.10.14 opencv-python --quiet

import types, mediapipe
mediapipe.tasks = types.SimpleNamespace()  # Patch for old tasks API

import mediapipe as mp
import cv2
import numpy as np

print("MediaPipe Version:", mp.__version__)
print("Ready to use! 🎉")


MediaPipe Version: 0.10.14
Ready to use! 🎉


In [ ]:
# ===============================
# Continuous Real-Time Sign Language Detection
# Using MediaPipe + ANN + LSTM + NLP
# ===============================

from google.colab import files, output as colab_output
import zipfile, os, cv2, numpy as np, time, base64
from IPython.display import display, HTML, Image as IPyImage, clear_output
import sys, types, mediapipe
mediapipe.tasks = types.SimpleNamespace()  # Prevent MediaPipe audio import errors
import mediapipe as mp

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM      # ✅ LSTM added
from tensorflow.keras.utils import to_categorical

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import ipywidgets as widgets
from collections import deque                        # ✅ for sequence buffer

# -------------------------------
# Step 0: Upload & Extract Dataset
# -------------------------------
print("Upload your dataset ZIP (Sign_Language_Images_Dataset.zip):")
uploaded = files.upload()  # Upload zip file

for fn in uploaded.keys():
    with zipfile.ZipFile(fn, 'r') as zip_ref:
        zip_ref.extractall('dataset')  # Extract to /content/dataset/

dataset_dir = 'dataset/Sign_Language_Images_Dataset'

# -------------------------------
# Step 1: Extract & Normalize Landmarks
# -------------------------------
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=1,
    min_detection_confidence=0.5
)

def extract_landmarks(img):
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(img_rgb)
    if results.multi_hand_landmarks:
        lm = results.multi_hand_landmarks[0]
        coords = []
        for point in lm.landmark:
            coords.extend([point.x, point.y])
        return coords
    else:
        return None

def normalize_landmarks(coords):
    coords = np.array(coords).reshape(-1, 2)
    wrist = coords[0]
    coords -= wrist
    max_dist = np.max(np.linalg.norm(coords, axis=1))
    if max_dist == 0:
        return coords.flatten()
    coords /= max_dist
    return coords.flatten()

# -------------------------------
# Step 2: Prepare Dataset
# -------------------------------
X, y = [], []
print("Processing images...")
for sign in os.listdir(dataset_dir):
    sign_folder = os.path.join(dataset_dir, sign)
    if os.path.isdir(sign_folder):
        for img_file in os.listdir(sign_folder):
            img_path = os.path.join(sign_folder, img_file)
            img = cv2.imread(img_path)
            if img is None:
                continue
            coords = extract_landmarks(img)
            if coords:
                coords = normalize_landmarks(coords)
                X.append(coords)
                y.append(sign)

X = np.array(X)
y = np.array(y)
print(f"Dataset prepared: {len(X)} samples, {len(set(y))} classes")

# Encode labels
le = LabelEncoder()
y_enc = le.fit_transform(y)
y_enc = to_categorical(y_enc, num_classes=len(le.classes_))

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y_enc, test_size=0.2, random_state=42
)

# -------------------------------
# Step 3: Build & Train ANN (MLP)
# -------------------------------
model = Sequential([
    Dense(128, activation='relu', input_shape=(X.shape[1],)),
    Dense(64, activation='relu'),
    Dense(len(le.classes_), activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    X_train, y_train,
    epochs=100, batch_size=8,
    validation_split=0.1, verbose=0
)

test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"✅ MLP Model training complete! Test Accuracy: {test_acc*100:.2f}%")

# -------------------------------
# Step 3B: Build Sequence Dataset for LSTM
# -------------------------------
SEQ_LEN = 15                 # number of frames in a sequence
FEATURE_DIM = X.shape[1]     # 42 (21 landmarks * 2)

# Synthetic sequences: repeat each static sample across SEQ_LEN timesteps
X_seq = np.repeat(X[:, np.newaxis, :], SEQ_LEN, axis=1)   # shape: (N, SEQ_LEN, FEATURE_DIM)
y_seq = y_enc                                             # same labels

print("LSTM sequence dataset shape:", X_seq.shape, y_seq.shape)

X_seq_train, X_seq_test, y_seq_train, y_seq_test = train_test_split(
    X_seq, y_seq, test_size=0.2, random_state=42
)

# -------------------------------
# Step 3C: Build & Train LSTM Model
# -------------------------------
lstm_model = Sequential([
    tf.keras.layers.Input(shape=(SEQ_LEN, FEATURE_DIM)),
    LSTM(64, return_sequences=False),
    Dense(64, activation='relu'),
    Dense(len(le.classes_), activation='softmax')
])

lstm_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history_lstm = lstm_model.fit(
    X_seq_train, y_seq_train,
    epochs=60,
    batch_size=8,
    validation_split=0.1,
    verbose=0
)

lstm_loss, lstm_acc = lstm_model.evaluate(X_seq_test, y_seq_test, verbose=0)
print(f"✅ LSTM Model training complete! Test Accuracy: {lstm_acc*100:.2f}%")

# -------------------------------
# Step 4: NLP Post-Processing + Suggestion Rules
# -------------------------------
CANONICAL_MAP = {
    "Hello": "Hello",
    "I love you": "I love you",
    "Thank you": "Thank you",
    "Yes": "Yes",
    "No": "No"
}

SUGGESTIONS_MAP = {
    "Hello": [
        "How can I help you?",
        "Hope you're doing well!"
    ],
    "Thank you": [
        "You're welcome!",
        "Glad I could help!"
    ],
    "Yes": [
        "Great, proceeding ahead.",
        "Okay, confirmed."
    ],
    "No": [
        "Would you like to try something else?",
        "Alright, noted."
    ],
    "I love you": [
        "❤️ That's very sweet!",
        "Love you too!"
    ]
}

def nlp_postprocess(label, confidence):
    """
    Light NLP:
      - map to canonical phrase
      - add simple punctuation/emojis
    """
    phrase = CANONICAL_MAP.get(label, label)

    if phrase == "Hello":
        phrase = "Hello!"
    elif phrase == "Thank you":
        phrase = "Thank you!"
    elif phrase == "I love you":
        phrase = "I love you ❤️"

    return phrase

def get_suggestions_for_phrase(phrase):
    """Return list of suggestion strings for the recognized phrase."""
    clean = phrase.replace(" ❤️", "")
    return SUGGESTIONS_MAP.get(clean, [])

# -------------------------------
# Step 5: Setup Webcam Widget
# -------------------------------
display(HTML("""
<style>
  #cambox { display:flex; gap:10px; align-items:center; }
</style>
<div id="cambox">
  <video autoplay playsinline id="video" width=360 height=270></video>
  <canvas id="canvas" width=360 height=270 style="display:none;"></canvas>
  <div>
    <button id="startBtn">Start</button>
    <button id="stopBtn">Stop</button>
    <p id="stats">Status: idle</p>
  </div>
</div>
<script>
var video = document.getElementById('video');
var canvas = document.getElementById('canvas');
var startBtn = document.getElementById('startBtn');
var stopBtn = document.getElementById('stopBtn');
var stats = document.getElementById('stats');
let streaming = false;
let stream;

async function startCam() {
  if (streaming) return;
  stream = await navigator.mediaDevices.getUserMedia({video:true, audio:false});
  video.srcObject = stream;
  streaming = true;
  stats.textContent = 'Status: streaming';
}

function stopCam() {
  if (!streaming) return;
  stream.getTracks().forEach(t => t.stop());
  streaming = false;
  stats.textContent = 'Status: stopped';
}

startBtn.onclick = startCam;
stopBtn.onclick = stopCam;

window.captureFrame = function() {
  if (!streaming) return "";
  var ctx = canvas.getContext('2d');
  ctx.drawImage(video, 0, 0, canvas.width, canvas.height);
  return canvas.toDataURL('image/jpeg', 0.8);
};
</script>
"""))

print("Webcam widget injected. Click 'Start' to enable webcam.")

# -------------------------------
# Step 6: Continuous Detection Loop (LSTM + NLP)
# -------------------------------
pred_output = widgets.Output()
display(pred_output)

prev_phrase = None
seq_buffer = deque(maxlen=SEQ_LEN)   # stores last SEQ_LEN landmark vectors

print("Starting real-time detection... Press 'Stop' in widget to exit.")
time.sleep(2)
print("⏳ Camera warming up... Please show a sign in front of the webcam.")

try:
    while True:
        # Capture frame from webcam
        b64 = colab_output.eval_js("window.captureFrame()")
        if not b64:
            time.sleep(0.2)
            continue

        img_bytes = base64.b64decode(b64.split(',')[-1])
        nparr = np.frombuffer(img_bytes, np.uint8)
        frame = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

        coords = extract_landmarks(frame)
        if coords:
            coords = normalize_landmarks(coords)
            seq_buffer.append(coords)

        # Need a full sequence for LSTM
        if len(seq_buffer) < SEQ_LEN:
            continue

        seq_input = np.array(seq_buffer).reshape(1, SEQ_LEN, FEATURE_DIM)

        # Use LSTM model for prediction
        pred_probs = lstm_model.predict(seq_input, verbose=0)[0]
        pred_idx = np.argmax(pred_probs)
        raw_label = le.inverse_transform([pred_idx])[0]

        # NLP post-processing
        phrase = nlp_postprocess(raw_label, confidence=float(np.max(pred_probs)))
        suggestions = get_suggestions_for_phrase(phrase)

        # Only update display if phrase changes
        if phrase != prev_phrase:
            with pred_output:
                clear_output(wait=True)
                display(IPyImage(data=img_bytes))
                print(f"🖐️ Detected sign: {phrase}")
                print(f"Confidence: {np.max(pred_probs):.2f}")
                if suggestions:
                    print("\n💡 Suggestions:")
                    for s in suggestions:
                        print(" -", s)
            prev_phrase = phrase

        time.sleep(0.3)

except KeyboardInterrupt:
    print("Stopped by user. See you next time! 👋")


Upload your dataset ZIP (Sign_Language_Images_Dataset.zip):


Saving Sign_Language_Images_Dataset_zip.zip to Sign_Language_Images_Dataset_zip (3).zip
Processing images...


/usr/local/lib/python3.12/dist-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Dataset prepared: 79 samples, 5 classes


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

Webcam widget injected. Click 'Start' to enable webcam.


Output()

Starting real-time detection... Press 'Stop' in widget to exit.
⏳ Camera warming up... Please show a sign in front of the webcam.


/usr/local/lib/python3.12/dist-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
/usr/local/lib/python3.12/dist-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
/usr/local/lib/python3.12/dist-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
/usr/local/lib/python3.12/dist-packages/google/protobuf/sy

Stopped by user. See you next time! 👋


Adding Front End using Gradio

In [ ]:
!pip install gradio --quiet
import gradio as gr

# ============================================
# 🔳 GRADIO UI FOR SIGN LANGUAGE DETECTION
# ============================================

def predict_uploaded_image(img):
    if img is None:
        return "No image uploaded", None

    # Convert from gradio (RGB) → OpenCV (BGR)
    img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    coords = extract_landmarks(img_bgr)
    if coords:
        coords = normalize_landmarks(coords)
        preds = model.predict(np.array([coords]), verbose=0)[0]
        max_idx = int(np.argmax(preds))
        max_prob = float(preds[max_idx])

        prediction = le.inverse_transform([max_idx])[0]
        return f"{prediction} (Confidence: {max_prob:.2f})", img
    else:
        return "No hand detected", img


# ----------- Gradio Layout ---------------
with gr.Blocks() as demo:
    gr.Markdown("<h1>🤟 Sign Language Detection (Upload Image)</h1>")
    gr.Markdown("Upload a photo of a hand sign to detect the gesture.")

    with gr.Row():
        img_input = gr.Image(type="numpy", label="Upload a hand image")
        img_output = gr.Image(label="Processed Image")

    prediction_text = gr.Textbox(label="Prediction")

    detect_button = gr.Button("Detect Sign")
    detect_button.click(
        predict_uploaded_image,
        inputs=[img_input],
        outputs=[prediction_text, img_output]
    )

demo.launch()


In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, classification_report
import numpy as np

# ------------------------------
# Convert predictions
# ------------------------------
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test, axis=1)

# ------------------------------
# MSE (Mean Squared Error)
# ------------------------------
mse = mean_squared_error(y_true, y_pred)

# ------------------------------
# RMSE (Root Mean Squared Error)
# ------------------------------
rmse = np.sqrt(mse)

# ------------------------------
# R² Score
# ------------------------------
r2 = r2_score(y_true, y_pred)

# ------------------------------
# Classification Report
# ------------------------------
report = classification_report(y_true, y_pred, target_names=le.classes_)

# ------------------------------
# Print Results
# ------------------------------
print("📌 PERFORMANCE METRICS")
print("------------------------------------")
print("Accuracy :", test_acc)
print("MSE      :", mse)
print("RMSE     :", rmse)
print("R² Score :", r2)
print("\nClassification Report:")
print(report)
